In [1]:
import pandas as pd
import html
import re
import requests
import urllib
import random

from urllib.parse import urlparse
from bs4 import BeautifulSoup
from urllib.parse import urlparse
from selenium import webdriver
from time import sleep

In [2]:
Main_site_url = 'https://www.chitai-gorod.ru'
author_names = ['Фрай М.', 'Хантер Э.', 'Емец Д.']
author_url_list = []
card_ids = []

In [3]:
def random_sleep(offset=1.5, length=4):
    sleep(random.random() * length + offset)

In [4]:
for author_name in author_names:
    name = urllib.parse.quote(author_name, encoding='windows-1251')
    author_url_list.append(Main_site_url + '/search/result.php?q=' + name + '&type=author')

In [5]:
for author_url in author_url_list:
    with webdriver.Safari() as driver:
        
        driver.get(author_url)
        random_sleep()
        
        items = driver.find_elements_by_class_name("count-result__value")
        for item in items:
            albums = int(item.text)
                
        while len(driver.find_elements_by_class_name("product-card")) != albums:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            elem = driver.find_element_by_css_selector(".js__show-more-cards")
            if elem:
                try:
                    elem.click()
                    random_sleep()
                except:
                    random_sleep()

        for i in driver.find_elements_by_class_name("product-card"):
            card_ids.append(i.get_attribute("data-product"))

        random_sleep(3)

In [6]:
def Insert_props(info, soup):
    tmp = soup.findAll('div', class_='product-prop')
        
    for i in tmp:
        soup1 = BeautifulSoup(str(i), 'html.parser')
        prop = soup1.find('div', class_='product-prop__title').text.strip()
        prop_info = soup1.find('div', class_='product-prop__value').text.strip()
        info[prop] = prop_info
    return info

In [7]:
def extract_book_info(card_id):

    book_url_head = Main_site_url + '/catalog/book/'
    book_url_tail = '/?watch_fromlist=search_result'
    book_url = book_url_head + card_id + book_url_tail
    
    book_html = requests.get(book_url).text
    soup = BeautifulSoup(book_html, 'html.parser')
    
    book_info = {
        'ID карточки': card_id,
        'Называние': soup.find('h1', class_='product__title js-analytic-product-title').text.strip(),
        'Автор': soup.find('a', class_='link product__author').text.strip(),
        'Рейтинг': float(soup.find('div', class_='rating').text.strip().split()[0]),
        'Голоса': int(soup.find('div', class_='rating').text.strip().split()[1][1: -1]),
        'Цена': int(soup.find('div', class_='price').text.strip()[:-1]) 
            if soup.find('div', class_='price') else None,
    }

    Insert_props(book_info, soup)

    book_info['Обложка'] = soup.find('img', itemprop='image').get('src')
    
    random_sleep(0, 1)
    return book_info

In [8]:
result = list(map(extract_book_info, card_ids))

df = pd.DataFrame(result)
df.sort_values(by='ID карточки', inplace=True)

In [9]:
df.head().T

,252,142,230,434,143
ID карточки,1002042,1004167,1004240,1006687,1007486
ID товара:,2607833,2608544,2608971,2611099,2611890
ISBN,9785699973705,9785171053802,9785001111900,9785699973712,9785179827924
Автор,Емец Д.,Фрай М.,Хантер Э.,Емец Д.,Фрай М.
Возраст,12+,18+,6+,12+,16+
Год издания,2017,2017,2017,2017,2017
Голоса,0,4,1,0,14
Издательство,Эксмо,АСТ,Абрис,Эксмо,АСТ
Кол-во страниц,416,288,350,416,416
Называние,Лед и пламя Тартара,Новая чайная книга,Расколотый прайд,Первый эйдос,Отдай мое сердце


In [10]:
with open('hw_4.csv', mode='w', encoding='utf-8') as f_csv:
    df.to_csv(f_csv, index=False)